In [ ]:
from fastai import *
from fastai.vision import *

In [ ]:
bs = 256

In [ ]:
df = pd.read_csv('../input/scene_classification/scene_classification/train.csv')

In [ ]:
tfms = get_transforms(do_flip = True)
data = (ImageDataBunch.from_csv("../input/scene_classification/scene_classification/train",  csv_labels='../train.csv') #Where to find the data? -> in path and its subfolders
        .split_by_rand_pct()              #How to split in train/valid? -> use the folders
        .label_from_df()            #How to label? -> depending on the folder of the filenames
        .add_test_folder(test_folder = '../test')              #Optionally add a test set (here default name is test)
        .transform(tfms, size = 128)       #Data augmentation? -> use tfms with a size of 64
        .databunch()).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.resnet152, metrics=[error_rate, accuracy], model_dir="/kaggle")

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1)

In [ ]:
# learn.freeze()
# learn.load('stage-1');
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
# learn.unfreeze()
learn.fit_one_cycle(5, max_lr = slice(5e-4, 1e-3))

In [ ]:
learn.save('stage-2')

In [ ]:
# learn.freeze()
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit(4, lr = 1.5e-6)

In [ ]:
learn.save('stage-3')

In [ ]:
# learn.unfreeze()
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, max_lr = slice(3e-5, 1e-4))

In [ ]:
# learn.lr_find()

In [ ]:
# learn.recorder.plot()

In [ ]:
# learn.save('stage-3')

In [ ]:
# learn.fit_one_cycle(6, max_lr = slice(2e-6, 9e-6))

In [ ]:
preds,_ = learn.get_preds(ds_type=DatasetType.Test)
predsPrime = list(np.array(preds))

In [ ]:
labelled_preds, labelled_predsPrime = [], []
for pred in preds:
    labelled_preds.append(int(np.argmax(pred)))
len(labelled_preds)

In [ ]:
import os
filenames = os.listdir('../input/scene_classification/scene_classification/test/')

In [ ]:
len(filenames) == len(labelled_preds)

In [ ]:
submission = pd.DataFrame(
    {'image_name': filenames,
     'label': labelled_preds,
    })

submissionPrime = pd.DataFrame(
    {'image_name': filenames,
     'label': predsPrime,
    })

In [ ]:
y_test = submission
y_testPrime = submissionPrime

image_name = list(submission['image_name'])
label = list(submission['label'])

image_namePrime = list(submissionPrime['image_name'])
labelPrime = list(submissionPrime['label'])

for i in range(len(submission)):
    if i%1000 == 0:
            print(i)
    for j in range(len(submission)):
        if j+1 < len(submission) and int(image_name[j][:-4]) > int(image_name[j+1][:-4]) :
            temp = image_name[j]
            image_name[j] = image_name[j+1]
            image_name[j+1] = temp
            
            tempPrime =  label[j]
            label[j] = label[j+1]
            label[j+1] = tempPrime
            
            temp = image_namePrime[j]
            image_namePrime[j] = image_namePrime[j+1]
            image_namePrime[j+1] = temp
            
            tempPrime =  labelPrime[j]
            labelPrime[j] = labelPrime[j+1]
            labelPrime[j+1] = tempPrime
            
y_test['image_name'] = pd.DataFrame(image_name)
y_test['label'] = pd.DataFrame(label)
y_test.to_csv('finalOne.csv')

y_testPrime['image_name'] = image_namePrime
y_testPrime['label'] = labelPrime
y_testPrime.to_csv('FINALONE.csv')